In [1]:
from pysentimiento import create_analyzer

"""
Reference: 
 title={pysentimiento: A Python Toolkit for Sentiment Analysis and SocialNLP tasks},
      author={Juan Manuel Pérez and Juan Carlos Giudici and Franco Luque},
      year={2021},
"""

analyzer = create_analyzer(task="emotion", lang="en", )

In [2]:
import snscrape.modules.twitter as sntwitter
import pandas as pd

# Creating list to append tweet data to
tweets_list = []

query='#doordash'
username='doordash'

start_date="2022-06-01"

search_term=query+" since:"+start_date

# Using TwitterSearchScraper to scrape data and append tweets to list
for i,tweet in enumerate(sntwitter.TwitterSearchScraper(search_term).get_items()):
    tweets_list.append([tweet.content, tweet.lang, tweet.retweetedTweet, tweet.user.username])
    
    
# Creating a dataframe from the tweets list above
tweets_df = pd.DataFrame(tweets_list, columns=['Text','Language', 'retweet', 'username'])

In [3]:
tweets_df = tweets_df[tweets_df['retweet'].isna()] #original posts (not retweets) have a null retweet value
tweets_df = tweets_df[tweets_df['username']!=username] #exclude tweets from the company 
tweets_df = tweets_df[tweets_df['Language']=='en'] #filter to only tweets with english as the language
test_df = tweets_df.loc[:,['Text']]#select the text column
test_df

,Text
0,@TMZ I bet #DoorDash and #GrubHub on 10! Right...
1,"$DASH 8-K DoorDash (14 Jul, 4:30pm)\n\n📰 E..."
2,#JaylandWalker\n#SuicideByCop\n\nMaybe if he h...
4,So I recorded bunch of videos for my page toda...
5,Why do ppl think that not tipping ur DoorDash ...
...,...
3570,Arrived to the restaurant and my order is gone...
3571,Gas went up another 16 cents over night. It's ...
3572,At this very moment….I’m wondering if 11.00 fo...
3573,#doordash 8.5 $ super busy???


In [4]:
def get_emotions(t): #get the emotions using pysentimento
    pred = analyzer.predict(t)
    return pred.output

In [5]:
test_df['emotions']=test_df['Text'].apply(get_emotions) #apply emotions function
test_df

,Text,emotions
0,@TMZ I bet #DoorDash and #GrubHub on 10! Right...,others
1,"$DASH 8-K DoorDash (14 Jul, 4:30pm)\n\n📰 E...",others
2,#JaylandWalker\n#SuicideByCop\n\nMaybe if he h...,disgust
4,So I recorded bunch of videos for my page toda...,joy
5,Why do ppl think that not tipping ur DoorDash ...,disgust
...,...,...
3570,Arrived to the restaurant and my order is gone...,others
3571,Gas went up another 16 cents over night. It's ...,others
3572,At this very moment….I’m wondering if 11.00 fo...,others
3573,#doordash 8.5 $ super busy???,others


In [6]:
test_df.to_csv('doordash_unlabeled.csv') #convert to csv